In [ ]:
#naimur rahaman
#intake 38 id 248

In [ ]:
import tensorflow 

import pandas as pd
import numpy as np
import os
import keras
import random
import cv2
import math
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [ ]:
#!unzip /content/drive/MyDrive/MangoData/NoiseFree_Original__augmented.zip

In [ ]:
batch=128
epoch=50
classes=20
directory="/content/NoiseFree_Original__augmented"
save_model_as="densenet_model.h5"

In [ ]:
model_d=DenseNet121(weights='imagenet',include_top=False, input_shape=(128, 128, 3)) 

x=model_d.output

x= GlobalAveragePooling2D()(x)
x= BatchNormalization()(x)
x= Dropout(0.5)(x)
x= Dense(1024,activation='relu')(x) 
x= Dense(512,activation='relu')(x) 
x= BatchNormalization()(x)
x= Dropout(0.5)(x)

preds=Dense(classes,activation='softmax')(x) #FC-layer

In [ ]:
model=Model(inputs=model_d.input,outputs=preds)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 134, 134, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 64, 64, 64)   9408        ['zero_padding2d[0][0]']         
                                                                                                  
 conv1/bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1/conv[0][0]']         

In [ ]:
for layer in model.layers[:-8]:
    layer.trainable=False
    
for layer in model.layers[-8:]:
    layer.trainable=True

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
data=[]
labels=[]
random.seed(42)
imagePaths = sorted(list(os.listdir(directory)))
random.shuffle(imagePaths)
print(imagePaths)

for img in imagePaths:
    path=sorted(list(os.listdir(directory+"/"+img)))
    for i in path:
        image = cv2.imread(directory+"/"+img+'/'+i)
        image = cv2.resize(image, (128,128))
        image = img_to_array(image)
        data.append(image)
        l = label = img
        labels.append(l)

['Shoot_mold_Leaf', 'Fruit_borer_Fruit', 'Normal_Leaf', 'Black_Rot_disease_Fruit', 'Leaf_Cutting_Weevil_Leaf', 'Normal_Fruit', 'Powdery_Mildew_Leaf', 'Scab_Leaf', 'Gall_midge_Leaf', 'Mysterious_cracking_Disease_Fruit', 'Scab_Fruit', 'Mealybug_Fruit', 'Anthracnose_Leaf', 'Mealybug_Leaf', 'Bacterial_Canker_Fruit', 'Red_Rust_Leaf', 'Giant_Mealybug_Leaf', 'Leaf_Coating_Mite_Leaf', 'Anthracnose_Fruit', 'Bacterial_Canker_Leaf']


In [ ]:
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)
mlb = LabelBinarizer()
labels = mlb.fit_transform(labels)
print(labels[0])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


In [ ]:
(xtrain,xtest,ytrain,ytest)=train_test_split(data,labels,test_size=0.4,random_state=42)
print(xtrain.shape, xtest.shape)

(19082, 128, 128, 3) (12722, 128, 128, 3)


In [ ]:
anne = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
checkpoint = ModelCheckpoint(save_model_as, verbose=1, save_best_only=True)

datagen = ImageDataGenerator(zoom_range = 0.2, horizontal_flip=True, shear_range=0.2)


datagen.fit(xtrain)
# Fits-the-model
history = model.fit_generator(datagen.flow(xtrain, ytrain, batch_size=batch),
               steps_per_epoch=xtrain.shape[0] //batch,
               epochs=epoch,
               verbose=2,
               callbacks=[anne, checkpoint],
               validation_data=(xtrain, ytrain))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/50

Epoch 00001: val_loss improved from inf to 0.23090, saving model to densenet_model.h5
149/149 - 107s - loss: 0.9177 - accuracy: 0.7226 - val_loss: 0.2309 - val_accuracy: 0.9363 - lr: 0.0010 - 107s/epoch - 718ms/step
Epoch 2/50

Epoch 00002: val_loss improved from 0.23090 to 0.06938, saving model to densenet_model.h5
149/149 - 87s - loss: 0.3592 - accuracy: 0.8802 - val_loss: 0.0694 - val_accuracy: 0.9839 - lr: 0.0010 - 87s/epoch - 585ms/step
Epoch 3/50

Epoch 00003: val_loss improved from 0.06938 to 0.03243, saving model to densenet_model.h5
149/149 - 87s - loss: 0.2426 - accuracy: 0.9181 - val_loss: 0.0324 - val_accuracy: 0.9936 - lr: 0.0010 - 87s/epoch - 585ms/step
Epoch 4/50

Epoch 00004: val_loss improved from 0.03243 to 0.02002, saving model to densenet_model.h5
149/149 - 87s - loss: 0.2001 - accuracy: 0.9309 - val_loss: 0.0200 - val_accuracy: 0.9951 - lr: 0.0010 - 87s/epoch - 583ms/step
Epoch 5/50

Epoch 00005: val_loss improved from 0.02002 to 0.01752, saving model t

In [ ]:
ypred = model.predict(xtest)

total = 0
accurate = 0
accurateindex = []
wrongindex = []

for i in range(len(ypred)):
    if np.argmax(ypred[i]) == np.argmax(ytest[i]):
        accurate += 1
        accurateindex.append(i)
    else:
        wrongindex.append(i)
        
    total += 1
    
print('Total-test-data;', total, '\taccurately-predicted-data:', accurate, '\t wrongly-predicted-data: ', total - accurate)
print('Accuracy:', round(accurate/total*100, 3), '%')

In [ ]:
label=['Shoot_mold_Leaf', 'Fruit_borer_Fruit', 'Normal_Leaf', 'Black_Rot_disease_Fruit', 'Leaf_Cutting_Weevil_Leaf', 'Normal_Fruit',
       'Powdery_Mildew_Leaf', 'Scab_Leaf', 'Gall_midge_Leaf', 'Mysterious_cracking_Disease_Fruit', 'Scab_Fruit', 'Mealybug_Fruit', 'Anthracnose_Leaf',
       'Mealybug_Leaf', 'Bacterial_Canker_Fruit', 'Red_Rust_Leaf', 'Giant_Mealybug_Leaf', 'Leaf_Coating_Mite_Leaf', 'Anthracnose_Fruit', 'Bacterial_Canker_Leaf']
imidx = random.sample(accurateindex, k=1)# replace with 'wrongindex'

nrows = 3
ncols = 3
fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True,figsize=(15, 12))

n = 0
for row in range(nrows):
    for col in range(ncols):
            ax[row,col].imshow(xtest[imidx[n]])
            ax[row,col].set_title("Predicted label :{}\nTrue label :{}".format(label[np.argmax(ypred[imidx[n]])], label[np.argmax(ytest[imidx[n]])]))
            n += 1

plt.show()

In [ ]:
Ypred = model.predict(xtest)

Ypred = np.argmax(Ypred, axis=1)
Ytrue = np.argmax(ytest, axis=1)

cm = confusion_matrix(Ytrue, Ypred)
plt.figure(figsize=(12, 12))
ax = sns.heatmap(cm, cmap="rocket_r", fmt=".01f",annot_kws={'size':16}, annot=True, square=True, xticklabels=label, yticklabels=label)
ax.set_ylabel('Actual', fontsize=20)
ax.set_xlabel('Predicted', fontsize=20)